# Sondera Harness + Custom Agent

**Use the Harness API directly for any agent architecture.**

This notebook shows how to use Sondera Harness with your own agent code, without a framework like LangGraph, ADK, or Strands. You have full control over when and how to call `adjudicate()`.

We'll build a complete agent loop that:
- Evaluates **user input** before sending to the model
- Evaluates **tool calls** before execution
- Demonstrates **BLOCK** and **STEER** patterns
- Shows **human-in-the-loop** approval for high-stakes actions

*This is a simulation: we evaluate what would be allowed, but no actual LLM calls are made.*

## Install

In [ ]:
%%capture
%pip install sondera-harness

## Define Agent Metadata

Define your agent's tools and metadata. Unlike framework integrations, you create the `Agent` and `Tool` objects directly for policy evaluation.

In [ ]:
import json

from sondera import Agent, Parameter, Tool

# Define tools with their parameter schemas
search_tool = Tool(
    name="search_database",
    description="Search the company database for records",
    parameters=[
        Parameter(name="query", description="Search query", type="string"),
        Parameter(name="table", description="Table to search", type="string"),
    ],
    parameters_json_schema=json.dumps(
        {
            "type": "object",
            "properties": {"query": {"type": "string"}, "table": {"type": "string"}},
            "required": ["query", "table"],
        }
    ),
)

update_tool = Tool(
    name="update_record",
    description="Update a record in the database",
    parameters=[
        Parameter(name="table", description="Table name", type="string"),
        Parameter(name="record_id", description="Record ID to update", type="string"),
        Parameter(name="data", description="New data as JSON string", type="string"),
    ],
    parameters_json_schema=json.dumps(
        {
            "type": "object",
            "properties": {
                "table": {"type": "string"},
                "record_id": {"type": "string"},
                "data": {"type": "string"},
            },
            "required": ["table", "record_id", "data"],
        }
    ),
)

delete_tool = Tool(
    name="delete_record",
    description="Delete a record from the database",
    parameters=[
        Parameter(name="table", description="Table name", type="string"),
        Parameter(name="record_id", description="Record ID to delete", type="string"),
    ],
    parameters_json_schema=json.dumps(
        {
            "type": "object",
            "properties": {
                "table": {"type": "string"},
                "record_id": {"type": "string"},
            },
            "required": ["table", "record_id"],
        }
    ),
)

agent_metadata = Agent(
    id="data_assistant",
    provider_id="custom",
    name="data_assistant",
    description="A database assistant that helps with data queries and updates.",
    instruction="Help users query and manage data. Always verify before destructive operations.",
    tools=[search_tool, update_tool, delete_tool],
)

print(
    f"Agent metadata for '{agent_metadata.name}' with {len(agent_metadata.tools)} tools: {[t.name for t in agent_metadata.tools]}"
)

## Define the Policy

Cedar policies control what the agent can do. The namespace matches the agent name.

In [ ]:
from cedar import PolicySet

policy_set = PolicySet("""
// Allow all searches
@id("allow-search")
permit(principal, action == data_assistant::Action::"search_database", resource);

// Allow updates by default (restricted below)
@id("allow-update")
permit(principal, action == data_assistant::Action::"update_record", resource);

// Allow deletes by default (restricted below)
@id("allow-delete")
permit(principal, action == data_assistant::Action::"delete_record", resource);

// Block updates to the 'users' table (sensitive)
@id("forbid-users-update")
forbid(principal, action == data_assistant::Action::"update_record", resource)
when {
  context has parameters &&
  context.parameters.table == "users"
};

// Block all deletes from 'users' table
@id("forbid-users-delete")
forbid(principal, action == data_assistant::Action::"delete_record", resource)
when {
  context has parameters &&
  context.parameters.table == "users"
};

// Block deletes from 'audit_log' (append-only)
@id("forbid-audit-delete")
forbid(principal, action == data_assistant::Action::"delete_record", resource)
when {
  context has parameters &&
  context.parameters.table == "audit_log"
};

// Block searches containing SQL injection patterns
@id("forbid-sql-injection")
forbid(principal, action == data_assistant::Action::"search_database", resource)
when {
  context has parameters &&
  (context.parameters.query like "*DROP*" ||
   context.parameters.query like "*DELETE*" ||
   context.parameters.query like "*;--*")
};
""")

print(f"Policy loaded with {len(policy_set)} rules")

## Initialize the Harness

In [ ]:
from sondera import CedarPolicyHarness
from sondera.harness.cedar.schema import agent_to_cedar_schema

schema = agent_to_cedar_schema(agent_metadata)
harness = CedarPolicyHarness(policy_set=policy_set, schema=schema)
await harness.initialize(agent=agent_metadata)

print("Harness initialized and ready")

## The Agent Loop

In a custom agent, you call `adjudicate()` at each stage of your loop:

```
User Input → [PRE_MODEL adjudicate] → Model → [POST_MODEL adjudicate]
                                         ↓
Tool Result ← [POST_TOOL adjudicate] ← Tool ← [PRE_TOOL adjudicate]
```

You decide whether to **BLOCK** (stop immediately) or **STEER** (feed back to model).

## Test Policy Evaluation

Let's test what the agent is allowed to do at the PRE_TOOL stage.

In [ ]:
from sondera import Decision, Role, Stage, ToolRequestContent

# ANSI color codes
GREEN = "\033[92m"
RED = "\033[91m"
YELLOW = "\033[93m"
RESET = "\033[0m"


def check(description: str, result, expected: Decision):
    color = GREEN if result.decision == Decision.ALLOW else RED
    status = result.decision.name
    icon = "\u2705" if result.decision == expected else "\u274c"
    print(f"{icon} {description} ({color}{status}{RESET})")


# =============================================================================
# ALLOWED OPERATIONS
# =============================================================================

# Normal search is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="search_database", args={"query": "name = 'John'", "table": "customers"}
    ),
)
check("Search customers table", result, Decision.ALLOW)

# Update non-sensitive table is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="update_record",
        args={
            "table": "orders",
            "record_id": "ORD001",
            "data": '{"status": "shipped"}',
        },
    ),
)
check("Update orders table", result, Decision.ALLOW)

# Delete from non-protected table is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="delete_record", args={"table": "temp_cache", "record_id": "CACHE001"}
    ),
)
check("Delete from temp_cache", result, Decision.ALLOW)

# =============================================================================
# BLOCKED OPERATIONS
# =============================================================================

# Update users table is blocked
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="update_record",
        args={"table": "users", "record_id": "USER001", "data": '{"role": "admin"}'},
    ),
)
check("Update users table (sensitive)", result, Decision.DENY)

# Delete from users is blocked
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="delete_record", args={"table": "users", "record_id": "USER001"}
    ),
)
check("Delete from users table", result, Decision.DENY)

# Delete from audit_log is blocked (append-only)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="delete_record", args={"table": "audit_log", "record_id": "LOG001"}
    ),
)
check("Delete from audit_log (append-only)", result, Decision.DENY)

# SQL injection attempt is blocked
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="search_database",
        args={"query": "1=1; DROP TABLE users;--", "table": "customers"},
    ),
)
check("SQL injection attempt", result, Decision.DENY)

print(f"\n{GREEN}Policy evaluations complete!{RESET}")

## BLOCK Pattern

Stop execution immediately when a policy denies an action. Use for security-critical operations where no retry is acceptable.

In [ ]:
async def execute_with_block(tool_call: dict) -> str:
    """Execute a tool call using BLOCK pattern."""
    result = await harness.adjudicate(
        Stage.PRE_TOOL,
        Role.MODEL,
        ToolRequestContent(tool_id=tool_call["name"], args=tool_call["args"]),
    )

    if result.is_denied:
        # BLOCK: Stop immediately with error
        return f"{RED}BLOCKED: {result.reason}{RESET}"

    # Simulate tool execution
    return f"{GREEN}Executed: {tool_call['name']}({tool_call['args']}){RESET}"


# Test BLOCK pattern
print("BLOCK Pattern Examples:")
print("-" * 40)

# This will succeed
result = await execute_with_block(
    {
        "name": "search_database",
        "args": {"query": "status = 'active'", "table": "orders"},
    }
)
print(f"Search orders: {result}")

# This will be blocked
result = await execute_with_block(
    {"name": "delete_record", "args": {"table": "users", "record_id": "USER001"}}
)
print(f"Delete user: {result}")

## STEER Pattern

Feed the denial back to the model so it can try a different approach. The agent learns from policy feedback and can self-correct.

In [ ]:
async def execute_with_steer(tool_call: dict, messages: list) -> tuple[str, list]:
    """Execute a tool call using STEER pattern."""
    result = await harness.adjudicate(
        Stage.PRE_TOOL,
        Role.MODEL,
        ToolRequestContent(tool_id=tool_call["name"], args=tool_call["args"]),
    )

    if result.is_denied:
        # STEER: Add feedback for the model to see
        feedback = f"Tool '{tool_call['name']}' was blocked by policy: {result.reason}. Please try a different approach."
        messages.append({"role": "system", "content": feedback})
        return f"{YELLOW}STEERED: {result.reason}{RESET}", messages

    # Simulate tool execution
    output = f"Result from {tool_call['name']}"
    messages.append({"role": "tool", "content": output})
    return f"{GREEN}Executed: {tool_call['name']}{RESET}", messages


# Test STEER pattern
print("STEER Pattern Examples:")
print("-" * 40)

conversation = [{"role": "user", "content": "Delete user USER001"}]

# First attempt - will be steered
result, conversation = await execute_with_steer(
    {"name": "delete_record", "args": {"table": "users", "record_id": "USER001"}},
    conversation,
)
print(f"Attempt 1: {result}")
print(f"  Messages now: {len(conversation)}")
print(f"  Last message: {conversation[-1]['content'][:80]}...")

# Model could now try a different approach (simulated)
print(f"\n{YELLOW}Model adjusts approach based on feedback...{RESET}")

# Second attempt - search instead of delete
result, conversation = await execute_with_steer(
    {"name": "search_database", "args": {"query": "id = 'USER001'", "table": "users"}},
    conversation,
)
print(f"Attempt 2: {result}")

## Human-in-the-Loop

For high-stakes denials, pause and ask a human for approval instead of blocking outright. This gives flexibility while maintaining oversight.

In [ ]:
# Simulated human approval (in real code, this would be a UI/API call)
SIMULATED_APPROVALS = {
    "delete_record:audit_log": False,  # Never approve audit deletions
    "delete_record:users": True,  # Approve user deletions with review
    "update_record:users": True,  # Approve user updates with review
}


def request_human_approval(tool_name: str, args: dict, reason: str) -> bool:
    """Request human approval for a denied action."""
    key = f"{tool_name}:{args.get('table', 'unknown')}"
    approved = SIMULATED_APPROVALS.get(key, False)
    status = f"{GREEN}APPROVED{RESET}" if approved else f"{RED}REJECTED{RESET}"
    print(f"  Human review for '{tool_name}' on '{args.get('table')}': {status}")
    return approved


async def execute_with_hitl(tool_call: dict) -> str:
    """Execute a tool call with human-in-the-loop for denials."""
    result = await harness.adjudicate(
        Stage.PRE_TOOL,
        Role.MODEL,
        ToolRequestContent(tool_id=tool_call["name"], args=tool_call["args"]),
    )

    if result.is_allowed:
        return f"{GREEN}Executed (policy allowed){RESET}"

    # Policy denied - escalate to human
    print(f"  {YELLOW}Policy denied: {result.reason}{RESET}")
    print(f"  {YELLOW}Escalating to human reviewer...{RESET}")

    if request_human_approval(tool_call["name"], tool_call["args"], result.reason):
        return f"{GREEN}Executed (human approved){RESET}"
    else:
        return f"{RED}Blocked (human rejected){RESET}"


# Test HITL pattern
print("Human-in-the-Loop Pattern Examples:")
print("-" * 40)

# Delete user - will be approved by human
print("\nDeleting user record:")
result = await execute_with_hitl(
    {"name": "delete_record", "args": {"table": "users", "record_id": "USER001"}}
)
print(f"Result: {result}")

# Delete audit log - will be rejected by human
print("\nDeleting audit log:")
result = await execute_with_hitl(
    {"name": "delete_record", "args": {"table": "audit_log", "record_id": "LOG001"}}
)
print(f"Result: {result}")

# Normal search - no human needed
print("\nSearching (no human review needed):")
result = await execute_with_hitl(
    {
        "name": "search_database",
        "args": {"query": "status = 'active'", "table": "orders"},
    }
)
print(f"Result: {result}")

## Try It Yourself

Experiment with different patterns:

- **Add new policies**: Try adding rules for specific record IDs or data patterns
- **Modify HITL logic**: Change which actions require human approval
- **Combine patterns**: Use BLOCK for critical actions, STEER for recoverable ones

In [ ]:
def show(description: str, result):
    color = GREEN if result.decision == Decision.ALLOW else RED
    print(f"{description}: {color}{result.decision.name}{RESET}")


# Try searching with different patterns
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="search_database",
        args={"query": "SELECT * FROM sensitive_data", "table": "reports"},
    ),
)
show("Search with SELECT keyword", result)

# Try another SQL injection pattern
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="search_database", args={"query": "' OR '1'='1", "table": "customers"}
    ),
)
show("SQL injection with OR", result)

# This one should be blocked (contains DELETE)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="search_database",
        args={"query": "status = 'DELETE_PENDING'", "table": "orders"},
    ),
)
show("Search containing DELETE word", result)

## Next Steps

- [Quickstart](https://docs.sondera.ai/quickstart/) - Simpler policy evaluation examples
- [Writing Policies](https://docs.sondera.ai/writing-policies/) - Cedar syntax and patterns
- [LangGraph Integration](https://docs.sondera.ai/integrations/langgraph/) - Use with LangChain/LangGraph
- [ADK Integration](https://docs.sondera.ai/integrations/adk/) - Use with Google ADK
- [Strands Integration](https://docs.sondera.ai/integrations/strands/) - Use with AWS Strands